In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Using TensorFlow backend.
C:\Users\Administrator\.conda\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Administrator\.conda\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Admi

In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [4]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [5]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\Administrator\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [6]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/200
47/47 [==============================] - 2s 44ms/step - loss: 2.1994 - acc: 0.0638
Epoch 2/200
47/47 [==============================] - 0s 467us/step - loss: 2.1275 - acc: 0.2128
Epoch 3/200
47/47 [==============================] - 0s 489us/step - loss: 2.0814 - acc: 0.2766
Epoch 4/200
47/47 [==============================] - 0s 553us/step - loss: 2.0265 - acc: 0.3191
Epoch 5/200
47/47 [==============================] - 0s 510us/step - loss: 1.9388 - acc: 0.2766
Epoch 6/200
47/47 [==============================] - 0s 447us/step - loss: 1.7161 - acc: 0.4894
Epoch 7/200
47/47 [==============================] - 0s 532us/step - loss: 1.6851 - acc: 0.5532
Epoch 8/200
47/47 [==============================] - 0s 595us/step - loss: 1.5583 - acc: 0.553

47/47 [==============================] - 0s 489us/step - loss: 0.0686 - acc: 1.0000
Epoch 76/200
47/47 [==============================] - 0s 468us/step - loss: 0.1275 - acc: 0.9362
Epoch 77/200
47/47 [==============================] - 0s 489us/step - loss: 0.0470 - acc: 0.9787
Epoch 78/200
47/47 [==============================] - 0s 447us/step - loss: 0.0400 - acc: 1.0000
Epoch 79/200
47/47 [==============================] - 0s 489us/step - loss: 0.1258 - acc: 0.9787
Epoch 80/200
47/47 [==============================] - 0s 447us/step - loss: 0.2057 - acc: 0.9787
Epoch 81/200
47/47 [==============================] - 0s 468us/step - loss: 0.0653 - acc: 0.9787
Epoch 82/200
47/47 [==============================] - 0s 447us/step - loss: 0.1012 - acc: 0.9787
Epoch 83/200
47/47 [==============================] - 0s 489us/step - loss: 0.0536 - acc: 0.9787
Epoch 84/200
47/47 [==============================] - 0s 510us/step - loss: 0.0321 - acc: 0.9787
Epoch 85/200
47/47 [=======================

47/47 [==============================] - 0s 468us/step - loss: 0.0084 - acc: 1.0000
Epoch 159/200
47/47 [==============================] - 0s 489us/step - loss: 0.0064 - acc: 1.0000
Epoch 160/200
47/47 [==============================] - 0s 489us/step - loss: 0.0226 - acc: 1.0000
Epoch 161/200
47/47 [==============================] - 0s 510us/step - loss: 0.0142 - acc: 1.0000
Epoch 162/200
47/47 [==============================] - 0s 447us/step - loss: 0.0315 - acc: 1.0000
Epoch 163/200
47/47 [==============================] - 0s 468us/step - loss: 0.0282 - acc: 1.0000
Epoch 164/200
47/47 [==============================] - 0s 468us/step - loss: 0.0116 - acc: 1.0000
Epoch 165/200
47/47 [==============================] - 0s 447us/step - loss: 0.0674 - acc: 0.9787
Epoch 166/200
47/47 [==============================] - 0s 595us/step - loss: 0.0120 - acc: 1.0000
Epoch 167/200
47/47 [==============================] - 0s 447us/step - loss: 0.0180 - acc: 1.0000
Epoch 168/200
47/47 [=============